<a href="https://colab.research.google.com/github/abmishra1234/LLM-Apps/blob/Development/GenerateSummaryForYoutubeVideo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# ! pip install pytube openai-whisper


In [ ]:
#!pip install openai

In [5]:
from pytube import YouTube
import whisper
import os
import openai

In [ ]:
from getpass import getpass
OA_token = getpass()
openai.api_key = OA_token

In [34]:
import traceback
# Print the traceback
print(traceback.format_exc())

NoneType: None



In [7]:
# Function to download YouTube video audio
def download_youtube_audio(youtube_url, output_path):
    yt = YouTube(youtube_url)
    audio_stream = yt.streams.get_audio_only()
    audio_stream.download(output_path=output_path, filename="audio.mp4")
    return os.path.join(output_path, "audio.mp4")  # pytube saves as mp4

In [8]:
# Function to transcribe audio using Whisper
def transcribe_audio(audio_file_path):
    # Load the Whisper model
    model = whisper.load_model("base")

    # Transcribe the audio
    result = model.transcribe(audio_file_path)
    return result['text']

In [36]:
# Function to generate a summary from the transcript
def generate_summary(transcript):
    # Sending the transcript to OpenAI's API
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",  # Replace with the appropriate model
        messages=[
            {
                "role": "system",
                "content": "Summarize the following transcript and highlight the actionable items and important points:"
            },
            {
                "role": "user",
                "content": transcript
            }
        ]
    )
    # Access the last message in the conversation for the summary
    summary = response['choices'][0]['message']['content']
    return summary.strip()

In [10]:
# function to perform the transcribe tasks
def transcribe_youtube_video(youtube_url, docs_path):
    # Ensure the docs_path exists
    if not os.path.exists(docs_path):
        os.makedirs(docs_path)

    # Download the audio from YouTube
    audio_file_path = download_youtube_audio(youtube_url, docs_path)

    # Transcribe the audio to text
    transcript = transcribe_audio(audio_file_path)

    # Save the transcript to a file
    transcript_file_path = os.path.join(docs_path, "transcript.txt")
    with open(transcript_file_path, 'w') as file:
        file.write(transcript)

    # Clean up the audio file if you no longer need it
    os.remove(audio_file_path)

    print("Transcription completed successfully!")
    return transcript_file_path

In [16]:
# Main function into the main workflow
def transcribe_and_summarize_youtube_video(youtube_url, docs_path):
    # Transcribe the YouTube video
    transcript_file_path = transcribe_youtube_video(youtube_url, docs_path)

    # Read the transcript from the file
    with open(transcript_file_path, 'r') as file:
        transcript = file.read()

    # Generate a summary of the transcript
    summary = generate_summary(transcript)

    # Save the summary to a file
    summary_file_path = os.path.join(docs_path, "summary.txt")
    with open(summary_file_path, 'w') as file:
        file.write(summary)

    print("Summary generation completed successfully!")
    return summary_file_path

In [12]:
# ! pip install ffmpeg

In [37]:
# Replace 'your_youtube_video_url' with the actual YouTube video URL
youtube_video_url = 'https://www.youtube.com/watch?v=JLmjPkR1eF4&t=64s'
docs_directory = "docs/youtube/"
summary_file = transcribe_and_summarize_youtube_video(youtube_video_url, docs_directory)
print(f"The summary has been saved to: {summary_file}")



/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcription completed successfully!
Summary generation completed successfully!
The summary has been saved to: docs/youtube/summary.txt


In [ ]:
#!openai migrate